In [2]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets # pytorch has a lot of datasets # https://docs.pytorch.org/vision/stable/datasets.html
import torchvision.transforms as transforms
from tqdm import tqdm

In [3]:
# Seed To Get Deterministic Behaviour
seed = 0
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
# Create Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes): # (28*28 = 784)
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.dropout = nn.Dropout(0.5)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        # x = self.softmax(self.fc2(x))
        return x

model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

torch.Size([64, 10])


In [5]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64 # 64
num_epochs = 3

In [7]:
# Data augmentation is good but not all data augmentation is good.
# mytransforms = transforms.Compose([
#     transforms.RandomVerticalFlip(p=1.0),
#     transforms.RandomHorizontalFlip(p=1.0),
#     transforms.ToTensor()
# ])

my_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,)) # normalize to improve performance
])

In [8]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, 
                               # transform=transforms.ToTensor(), 
                               transform=my_transforms,
                               download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) # shuffle is important

test_dataset = datasets.MNIST(root='dataset/', train=False, 
                              # transform=transforms.ToTensor(),
                              transform=my_transforms,
                              download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [9]:
# Initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [10]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Take out a single batch (Before training a model, make sure to overfit a single batch first )
# data, targets = next(iter(train_loader))

In [12]:
# for epoch in range(num_epochs):
#     print(f"Epoch [{epoch+1}/{num_epochs}]")
#     # for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
#         # Get data to cuda if possible
#     data = data.to(device=device)
#     targets = targets.to(device=device)

#     # Get to correct shape
#     data = data.reshape(data.shape[0], -1)

#     # forward
#     scores = model(data)
#     loss = criterion(scores, targets)
#     print(loss)

#     # backward
#     optimizer.zero_grad()
#     loss.backward()

#     # gradient descent or adam step
#     optimizer.step()

In [13]:
# Train network

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad() # very important @ no accumulated gradients
        loss.backward()
         # Gradient clipping (try-except for compatibility)
        try:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        except AttributeError:
            torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=1.0)

        # gradient descent or adam step
        optimizer.step()

100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:15<00:00, 60.46it/s]


In [14]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            scores.max()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    # model.train()

# check_accuracy(train_loader, model)
# check_accuracy(test_loader, model)

In [15]:
check_accuracy(test_loader, model)

# Always remember to do this
model.eval()
check_accuracy(train_loader, model)
model.train()

Checking accuracy on test data
Got 9482 / 10000 with accuracy 94.82
Checking accuracy on training data
Got 57064 / 60000 with accuracy 95.11


NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)